In [1]:
#This is the notebook for testing the pipeline before pushing any changes
import localnmf 
from localnmf import superpixel_analysis_ring
import os
import numpy as np
import scipy
import scipy.sparse
import torch_sparse
import torch

from localnmf import visualization
from localnmf.visualization import standard_demix_vid_m
import scipy.sparse
import scipy
import matplotlib.pyplot as plt
from localnmf.signal_demixer import SignalDemixer
%load_ext line_profiler

In [2]:
input_file = "../datasets/decomposition.npz"
data = np.load(input_file, allow_pickle=True)

In [3]:
U_sparse = scipy.sparse.csr_matrix(
        (data['U_data'], data['U_indices'], data['U_indptr']),
        shape=data['U_shape']
    ).tocsr()
order = data.get('fov_order', np.array("C")).item()
shape = data['U_shape']
d1,d2 = data['fov_shape']

R = data['R']
s = data['s']
Vt = data['Vt']
T = Vt.shape[1]
mean_img = data['mean_img']
noise_var_img = data['noise_var_img']
device='cuda'

# Initialize PMD Object which stores the state of the demixing procedure

In [4]:
pmd_video = SignalDemixer(U_sparse, R, s, Vt, (d1, d2, T), mean_img, noise_var_img, data_order=order, device=device)

In [5]:
# First pass initialization of neural signals

In [2]:
num_plane = 1
cut_off_point = 0.9
residual_cut = 0.1
length_cut = 3
th = 2
pseudo_2 = 0.1

pmd_video.initialize_signals_superpixels(num_plane, cut_off_point, residual_cut, length_cut, th, pseudo_2, \
                                       text =True, plot_en = True)

# Run the demixing procedure

In [3]:
localnmf_params = {
        'init': ['lnmf', 'lnmf', 'lnmf', 'lnmf'],
        'pseudo_2':[0.1, 0.1, 0.1, 0.1],
        'corr_th_fix':0.55,
        'switch_point':5,
        'corr_th_fix_sec':0.7,
        'corr_th_del':0.2,
        'merge_corr_thr':0.7,
        'merge_overlap_thr':0.7,
        'r':20,
        'residual_cut':[0.5, 0.6, 0.6, 0.6],
        'num_plane': 1,
        'patch_size': [100,100],
        'maxiter': 25,
        'update_after':8, 
        'plot_en': False,
        'skips':0,
}

corr_th_fix=localnmf_params['corr_th_fix'] 
switch_point = localnmf_params['switch_point']
corr_th_fix_sec = localnmf_params['corr_th_fix_sec']
corr_th_del = localnmf_params['corr_th_del']

merge_corr_thr= localnmf_params['merge_corr_thr']
merge_overlap_thr= localnmf_params['merge_overlap_thr']
ring_radius =  localnmf_params['r']
pseudo_2 = localnmf_params['pseudo_2']


residual_cut = localnmf_params['residual_cut']
num_plane= localnmf_params['num_plane']
patch_size= localnmf_params['patch_size'] 
plot_en = localnmf_params['plot_en'] 
maxiter= localnmf_params['maxiter']
update_after = localnmf_params['update_after']
skips= localnmf_params['skips'] 
plot_debug = False
denoise = [False for i in range(maxiter)]



from localnmf.superpixel_analysis_ring import update_AC_bg_l2_Y_ring_lowrank
from localnmf.signal_demixer import SignalDemixer
from localnmf.signal_demixer import merge_components, vcorrcoef_resid

from localnmf.regression_update import spatial_update_HALS, temporal_update_HALS, project_U_HALS

with torch.no_grad():

    %lprun -f PMDVideo.support_update_prune_elements_apply_mask a, c, b, W, res, corr_img_all_r, num_list = superpixel_analysis_ring.update_AC_bg_l2_Y_ring_lowrank(pmd_video, maxiter, corr_th_fix, \
                                                                                                           corr_th_fix_sec, corr_th_del, switch_point, skips, \
                                                                                                           merge_corr_thr, merge_overlap_thr, ring_radius, denoise=denoise,\
                                                                                                           plot_en=plot_en, plot_debug=plot_debug, update_after=update_after)
    W_final = W.create_complete_ring_matrix(a)
    fin_rlt = {'U_sparse': pmd_video.u_sparse.cpu().to_scipy(layout='csr'), 'R': pmd_video.r.cpu().numpy(), 'V': pmd_video.v.cpu().numpy(), 'a':a, 'c':c, 'b':b, "W":W_final, \
               'res':res, 'corr_img_all_r':corr_img_all_r, 'num_list':num_list, 'data_order': pmd_video.data_order, 'data_shape':pmd_video.shape};




In [4]:
for k in range(c.shape[1]):
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(a[:, k].reshape((60, 80), order=pmd_video.data_order))
    ax[1].plot(c[:, k])
    plt.show()